In [1]:
# partially copied from https://github.com/YBIGTA/facebigta/tree/master/PreProcessing

In [2]:
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import imutils
import dlib
import cv2
import os
import sys
import inspect
import matplotlib.pyplot as plt

In [3]:
from keras.models import Model
from keras.models import load_model
from keras.layers import Input, Activation, add, Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
from keras.preprocessing import image as keras_image

Using TensorFlow backend.


In [4]:
from pathlib import Path
import numpy as np
from keras.utils.data_utils import get_file

In [66]:
DATASET_NAME = 'kpop_profile2'
ROOT_DIR = '/Users/jerry.chi/Documents/preprocess_face'
DATASET_DIR = f'{ROOT_DIR}/{DATASET_NAME}'
PREDICTOR_PATH = ROOT_DIR + '/shape_predictor_68_face_landmarks.dat'
SOURCE_FOLDER = f'{DATASET_DIR}/raw' # images scraped using scrape_kpop_images.ipynb
DES_FOLDER = f'{DATASET_DIR}/processed'

In [67]:
GENDER_MODEL_PATH = '/Users/jerry.chi/git/CADL/face_classification/trained_models/gender_models/simple_CNN.81-0.96.hdf5' # from https://modeldepot.io/harshsikka/gender-classification
GENDER_MODEL_PATH2 = '/Users/jerry.chi/git/CADL/face_classification/trained_models/gender_models/gender_mini_XCEPTION.21-0.95.hdf5'

In [68]:
# create folders if they don't exist
for dir_name in [f'{ROOT_DIR}/{DATASET_NAME}', SOURCE_FOLDER, DES_FOLDER]:
    try:
        os.mkdir(dir_name)
        os.chmod(dir_name, 0o777)
    except:
        pass

In [ ]:
# Below gender classification from https://modeldepot.io/harshsikka/gender-classification

In [8]:
def load_image(image_path, color_mode=None, target_size=None):
    pil_image = keras_image.load_img(image_path, color_mode=color_mode, target_size=target_size)
    return keras_image.img_to_array(pil_image)

In [9]:
#gender_offsets = (30, 60)
gender_offsets = (10, 10)

In [30]:
gender_classifier = load_model(GENDER_MODEL_PATH, compile=False)
gender_classifier2 = load_model(GENDER_MODEL_PATH2, compile=False)

In [69]:
def format_np_image(x, v2=True, gender_target_size=None):
    if not gender_target_size:
        gender_target_size = gender_classifier.input_shape[1:3]
    x = cv2.resize(x, (gender_target_size))      
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    x = np.expand_dims(x, 0)
    if len(x.shape) == 3:
        x= np.expand_dims(x, 3)
    return x

In [74]:
start_file_number = 1
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)
fa = FaceAligner(predictor, desiredFaceWidth=100)

input_files = [os.path.join(dp,f) for dp, dn, fn in os.walk(os.path.expanduser(SOURCE_FOLDER)) for f in fn if (('jpg' in f) or ('jpeg' in f) or ('JPG' in f) or ('JPEG' in f) or ('png' in f)) and int(f.split('.')[0])>=start_file_number]
print("number of input files: {}".format(len(input_files)))

# loop over the face detections
for input_file in input_files:
        print('processing file: {}'.format(input_file))
        image = cv2.imread(input_file)
        height, width, channels = image.shape
        if (height < 300) or (width < 300):
            print('skipping image due to insufficient resolution.')
            continue
        image = imutils.resize(image, width=800)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # show the original input image and detect faces in the grayscale
        # image
        rects = detector(gray, 1) # 1 is the num of times to upsample https://towardsdatascience.com/cnn-based-face-detector-from-dlib-c3696195e01c
        for rect_i, rect in enumerate(rects):
                try:
                    # extract the ROI of the *original* face, then align the face
                    # using facial landmarks
                    src_file_basename, src_file_ext = tuple(input_file.split("/")[-1].split('.'))
                    out_file = os.path.join(DES_FOLDER,f'{src_file_basename}_{rect_i}.{src_file_ext}')

                    (x, y, w, h) = rect_to_bb(rect)
                    if (w < 70) or (h < 70):
                        print('skipping face because insufficient resolution.')
                        continue
                    #face_resized = imutils.resize(image[y:y + h, x:x + w], width=100)
                    face_image = image[y:y+h, x:x+w]
                    face_image_gray = gray[y:y + h, x:x + w]
                    # detect gender
                    face_image_formatted = format_np_image(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB), gender_target_size=gender_classifier.input_shape[1:3])
                    gender_prediction = gender_classifier.predict(face_image_formatted)
                    face_image_gray_formatted = format_np_image(face_image_gray, gender_target_size=gender_classifier2.input_shape[1:3])
                    gender_prediction2 = gender_classifier2.predict(face_image_gray_formatted)
                    if not ( (gender_prediction[0][0] > 0.72) and (gender_prediction2[0][0] > 0.72) ):
                            print('skipping face because not sure if female')
                            continue # only keep the image if the picture  female with high certainty
                    faceAligned = fa.align(image, gray, rect)
                    # write to file
                    #               if os.path.isfile(out_file):
                            #file_name = out_file.split("\\")[-1]
                            #print(file_name.split(".")[0]+"0"+file_name.split(".")[1])
                            #out_file = os.path.join(des_path, file_name.split(".")[0]+"(0)."+file_name.split(".")[-1])
                    cv2.imwrite(out_file,faceAligned)
                    print('wrote file: {}'.format(out_file))
                except:
                    print('some weird error?')
                    continue


number of input files: 999
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000001.jpg
skipping face because not sure if female
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000002.JPG
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000003.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000004.jpg
skipping face because not sure if female
skipping face because not sure if female
wrote file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/processed/000004_2.jpg
skipping face because insufficient resolution.
skipping face because insufficient resolution.
skipping face because insufficient resolution.
skipping face because insufficient resolution.
skipping face because insufficient resolution.
skipping face because insufficient resolutio

wrote file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/processed/000037_0.jpg
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000038.jpg
skipping face because not sure if female
skipping face because insufficient resolution.
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000039.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000040.jpg
skipping face because insufficient resolution.
skipping face because insufficient resolution.
skipping face because insufficient resolution.
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000041.jpg
skipping face because not sure if female
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000042.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profi

skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000071.jpg
wrote file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/processed/000071_0.jpg
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000072.jpg
skipping face because insufficient resolution.
skipping face because insufficient resolution.
skipping face because insufficient resolution.
skipping face because insufficient resolution.
skipping face because insufficient resolution.
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000073.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000074.jpg
skipping face because insufficient resolution.
skipping face because insufficient resolution.
skipping face because insufficient resolution.
skipping face because insufficient resolution.
skipping face because insufficient resolution.


wrote file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/processed/000108_0.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000109.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000110.jpg
skipping face because not sure if female
skipping face because not sure if female
skipping face because not sure if female
skipping face because not sure if female
wrote file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/processed/000110_4.jpg
skipping face because not sure if female
wrote file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/processed/000110_6.jpg
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000111.jpg
wrote file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/processed/000111_0.jpg
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/

wrote file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/processed/000147_0.jpg
wrote file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/processed/000147_1.jpg
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000148.jpg
wrote file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/processed/000148_0.jpg
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000149.jpg
skipping face because not sure if female
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000150.jpg
skipping face because not sure if female
skipping face because not sure if female
skipping face because not sure if female
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000151.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000

...

wrote file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/processed/000780_0.jpg
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000781.jpeg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000782.jpg
skipping face because not sure if female
skipping face because not sure if female
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000783.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000784.png
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000785.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000786.jpg
skipping face because not sure if female
skipping face because not sure if female
proc

skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000826.jpg
wrote file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/processed/000826_0.jpg
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000827.jpg
skipping face because not sure if female
wrote file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/processed/000827_1.jpg
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000828.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000829.jpg
skipping face because not sure if female
skipping face because not sure if female
skipping face because insufficient resolution.
skipping face because not sure if female
skipping face because insufficient resolution.
skipping face because insufficient resolution.
skipping face because insufficient resolution.
skipping face because not 

skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000870.jpg
wrote file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/processed/000870_0.jpg
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000871.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000872.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000873.jpg
skipping face because insufficient resolution.
skipping face because not sure if female
skipping face because not sure if female
skipping face because not sure if female
skipping face because insufficient resolution.
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000874.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documen

skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000916.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000917.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000918.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000919.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000920.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000921.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000922.jpg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/prepr

processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000975.jpg
skipping face because not sure if female
skipping face because not sure if female
skipping face because not sure if female
skipping face because not sure if female
skipping face because not sure if female
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000976.jpeg
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000977.jpeg
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000978.jpg
skipping face because not sure if female
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000979.jpeg
skipping face because not sure if female
skipping face because not sure if female
processing file: /Users/jerry.chi/Documents/preprocess_face/kpop_profile2/raw/000980.jpeg
skipping fa